In [ ]:
# Import necessary libraries
import face_recognition
import cv2
import numpy as np
import csv
from datetime import datetime


In [ ]:
# Initialize the video capture object for the default camera
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)


In [ ]:
# Load known faces and their names
known_face_encodings = []
known_face_names = []

# Add your known faces here
for person in ["harpitha", "elon"]:
    image_path = f"C:\Users\Harpitha\OneDrive\Documents\z.DOCSSS\IMP\Harpitha.jpg"
    image = face_recognition.load_image_file(image_path)
    encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(encoding)
    known_face_names.append(person)

# Create a copy for expected students
expected_students = known_face_names.copy()


In [ ]:
# Initialize variables for face locations and encodings
face_locations = []
face_encodings = []

# Get the current date
now = datetime.now()
current_date = now.strftime("%d-%m-%y")


In [ ]:
# Open a CSV file for writing attendance
attendance_file = f"{current_date}.csv"
with open(attendance_file, "w+", newline="") as f:
    writer = csv.writer(f)


In [ ]:
    while True:
        # Capture video frame
        _, frame = cap.read()

        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Recognize faces in the frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        # Process each face found
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            face_distance = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distance)

            if matches[best_match_index]:
                name = known_face_names[best_match_index]

                # Display attendance status on the video feed
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, f"{name} PRESENT", (10, 100), font, 1.5, (255, 0, 0), 3)

                # Record attendance
                if name in expected_students:
                    expected_students.remove(name)
                    current_time = now.strftime("%H:%M:%S")
                    writer.writerow([name, current_time])

        # Show the video feed
        cv2.imshow("Attendance System", frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break


In [ ]:
# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
from openpyxl import Workbook, load_workbook

# Define the Excel file name
excel_file = f"{current_date}_attendance.xlsx"

# Check if the file already exists, else create a new one
try:
    workbook = load_workbook(excel_file)
    sheet = workbook.active
except FileNotFoundError:
    workbook = Workbook()
    sheet = workbook.active
    # Add headers to the sheet
    sheet.append(["Name", "Time"])

# Function to write data to Excel
def write_to_excel(name, time):
    sheet.append([name, time])
    workbook.save(excel_file)
